In [3]:
from src.util_tools import compute_cross_entropy, compute_ortho_loss
from tools.project import INPUT_PATH, LOGS_PATH, OUTPUT_PATH, MODELS_PATH

import audiocraft
from audiocraft.models import MusicGen
from audiocraft.utils.notebook import display_audio
import torch
from torch.onnx.symbolic_opset9 import tensor
from torchviz import make_dot
import typing as tp
from audiocraft.modules.conditioners import ConditioningAttributes
import tqdm
import torch
from audiocraft.data.audio import audio_read, audio_write
from audiocraft.data.audio_utils import convert_audio_channels, convert_audio
import umap
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from torch.utils.tensorboard import SummaryWriter
from sklearn.decomposition import PCA
from random import shuffle
from torch.utils.data import TensorDataset, random_split, DataLoader
from audioldm_eval.metrics.fad import FrechetAudioDistance
import os
import contextlib
import io
import torchaudio
def count_directories(path):
    import os
    if not os.path.exists(path):
        return 0
    return sum(os.path.isdir(os.path.join(path, entry)) for entry in os.listdir(path))

letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
EXP_NUM = count_directories(LOGS_PATH('textual-inversion'))+1
EXAMPLES_LEN = 5
BATCH_SIZE = 5
N_TOKENS = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

True

In [2]:
model = MusicGen.get_pretrained('facebook/musicgen-style')


config.json:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

configuration_MERT.py:   0%|          | 0.00/5.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/m-a-p/MERT-v1-95M:
- configuration_MERT.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_MERT.py:   0%|          | 0.00/18.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/m-a-p/MERT-v1-95M:
- modeling_MERT.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

/home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


compression_state_dict.bin:   0%|          | 0.00/236M [00:00<?, ?B/s]

/home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [23]:
model.set_generation_params(
    duration=8, # generate 8 seconds, can go up to 30
    use_sampling=True, 
    top_k=250,
    cfg_coef=3., # Classifier Free Guidance coefficient 
    cfg_coef_beta=8., # double CFG is necessary for text-and-style conditioning
                   # Beta in the double CFG formula. between 1 and 9. When set to 1 it is equivalent to normal CFG. 
                   # When we increase this parameter, the text condition is pushed. See the bottom of https://musicgenstyle.github.io/ 
                   # to better understand the effects of the double CFG coefficients. 
)

model.set_style_conditioner_params(
    eval_q=2, # integer between 1 and 6
              # eval_q is the level of quantization that passes
              # through the conditioner. When low, the models adheres less to the 
              # audio conditioning
    excerpt_length=4.5, # the length in seconds that is taken by the model in the provided excerpt, can be                 
                       # between 1.5 and 4.5 seconds but it has to be shortest to the length of the provided conditioning
    )

In [25]:
concept="ichika"
melody, sr = torchaudio.load(INPUT_PATH('textual-inversion-v3', 'data', 'valid', f'{concept}', 'audio', "music_p1.wav"))
res=model.generate_with_chroma([f"90s rock song with electric guitar and heavy drums"]*3, melody[None].expand(3, -1, -1), sr, progress=True)
display_audio(melody,sr )
display_audio(res,model.cfg.sample_rate )